In [61]:
# inport
import numpy as np
import cv2
import pickle

# Traffic sign labels

In [62]:
labelData = {
    '101' :  'Prohibited road',
    '102' :  'Prohibition of going in the opposite direction',
    '103a' :  'No cars allowed',
    '103b' :  'Cars are prohibited from turning right',
    '103c' :  'Cars are prohibited from turning left',
    '104' :  'Motorcycles are prohibited',
    '105' :  'Prohibited cars and motorbikes',
    '106a' :  'Prohibited trucks',
    '106b' :  'Allowed load capacity of the truck.',
    '106c' :  'Vehicles prohibited from carrying dangerous goods',
    '107' :  'Passenger cars and trucks are prohibited',
    '108' :  'Prohibition of cars, tractors pulling trailers and semi-trailers',
    '109' :  'Tractors are prohibited',
    '110a' :  'Bicycles are prohibited',
    '110b' :  'Prohibition of bicycles',
    '111a' :  'Motorcycles are prohibited',
    '111b' :  'Prohibited auto-rickshaws',
    '111c' :  'Prohibiting motor vehicles',
    '111d' :  'Non-motorized 3-wheeled vehicles are prohibited',
    '112' :  'No pedestrians allowed',
    '113' :  'Vehicles with people pulling or pushing are prohibited',
    '114' :  'Forklifts and tractors are prohibited',
    '115' :  'Vehicle weight restriction',
    '116' :  'Limit weight on vehicle axle',
    '117' :  'Height restriction',
    '118' :  'Horizontal restriction',
    '119' :  'Car length restriction',
    '120' :  'Restriction on the length of cars, tractors pulling trailers, and semi-trailers',
    '121' :  'Minimum distance between 2 vehicles',
    '122' :  'Stop',
    '123a' :  'Prohibited left turn',
    '123b' :  'Prohibited right turn',
    '124a' :  'U-turn is prohibited',
    '124b' :  'Cars are prohibited from turning around',
    '124c' :  'Left turns and U-turns are prohibited',
    '124d' :  'Right turns and U-turns are prohibited',
    '124e' :  'Cars are prohibited from turning left and making U-turns',
    '124f' :  'Cars are prohibited from turning right and making U-turns',
    '125' :  'No overtaking',
    '126' :  'Overtaking trucks are prohibited',
    '127' :  'Maximum allowed speed',
    '128' :  'Prohibition of using horns',
    '129' :  'Check',
    '130' :  'Parking is prohibited',
    '131a' :  'Parking prohibited',
    '131b' :  'Parking prohibited on odd days',
    '131c' :  'Parking prohibited on even days',
    '132' :  'Give way to motor vehicles traveling in the opposite direction on the narrow road',
    '136' :  'Prohibition of going straight',
    '137' :  'Prohibited left and right turns',
    '138' :  'Prohibition of going straight and turning left',
    '139' :  'Prohibition of going straight and turning right',
    '140' :  'Prohibited agricultural vehicles',

    # My label
    '141' :  'Ban cyclos and rickshaws',
    '142' :  'Ban trucks and passenger vehicles',
    '143' :  'Taxi ban',
    '144' :  'Ban auto-rickshaws and motorbikes',
    '145' :  'Ban trucks from 4 tons and passenger cars from 16 seats',
}

# Manual training

In [63]:
def Manual_training():
    dataset = []
    for key in labelData:
        i = 1
        image_gray = cv2.imread(f"DataTraning/image({key}_{i}).png", cv2.IMREAD_GRAYSCALE)
        while image_gray is not None:
            # Áp dụng phép lọc Gaussian để làm mờ ảnh gốc
            input_blur = cv2.GaussianBlur(image_gray, (5, 5), 0)
            try:
                if labelData[key] is not None:
                    dataset.append({"image" : input_blur,"label" : str(labelData[key])})
                    i += 1
                    image_gray = cv2.imread(f"DataTraning/image({key}_{i}).png", cv2.IMREAD_GRAYSCALE)
            except Exception as e:
                print(f"Lỗi xảy ra: {e}")
                # Tiếp tục thực hiện các tác vụ khác sau khi bỏ qua lỗi
    return dataset

In [64]:
# # Ghi dữ liệu vào file
# with open('data.pickle', 'ab') as file:
#     pickle.dump(Manual_training(), file)

In [65]:
# Đọc dữ liệu từ file
with open('data.pickle', 'rb') as file:
    dataset = pickle.load(file)

# Detection

In [66]:
def red_color_detection(image):

    # Chuyển đổi ảnh sang không gian màu HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Xác định vùng màu của biển báo cấm trong không gian màu HSV
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_image, lower_red, upper_red)

    lower_red = np.array([170, 120, 70])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_image, lower_red, upper_red)

    # Kết hợp các vùng màu
    mask = mask1 + mask2

    # Loại bỏ nhiễu và kết nối các đối tượng
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.dilate(mask, kernel, iterations = 1)

    # Trả về một hình ảnh đã được nhận diện màu các màu đỏ trong hình ảnh.
    return mask

In [67]:
def coordinates_detect(image):
    color_detection = red_color_detection(image)

    # Làm mờ hình ảnh để giảm nhiễu
    blurred = cv2.GaussianBlur(color_detection, (5, 5), 0)

    # Áp dụng phát hiện biên Canny để tìm cạnh
    edges = cv2.Canny(blurred, 50, 150)

    # Sử dụng phương pháp Hough Circles để nhận diện các hình tròn
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=40, param1=60, param2=30, minRadius=13, maxRadius=100)
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        return circles
    else:
        return None

In [68]:
def meaning_of_signs(image):
    # Chuyển đổi sang ảnh xám
    input_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Áp dụng phép lọc Gaussian để làm mờ ảnh gốc
    input_blur = cv2.GaussianBlur(input_gray, (5, 5), 0)

    meanMax = 0.0
    label = ''
    for data in dataset:
        image = data['image']

        # Lấy kích thước của hai hình ảnh gốc
        height1, width1 = image.shape[:2]
        height2, width2 = input_blur.shape[:2]
        # Xác định kích thước mới cho các hình ảnh
        new_width = min(width1, width2)
        new_height = min(height1, height2)

        # Điều chỉnh kích thước của hai hình ảnh
        input_blur = cv2.resize(input_blur, (new_width, new_height))
        image = cv2.resize(image, (new_width, new_height))

        result = cv2.matchTemplate(input_blur, image, cv2.TM_CCOEFF_NORMED)
        mean = np.mean(result)
        if(mean >= meanMax):
            label = data['label']
            meanMax = mean
    if(meanMax < 0.05):
        return ''
    return label

In [69]:
def draw_rectangle(image, index):
    coordinates = coordinates_detect(image)
    color_rectangle = [
                        (0, 255, 0),
                        (0, 255, 255),
                        (255, 0, 0),
                        (255, 0, 255),
                        (147, 20, 255),
                    ]
    i_color = 0
    if coordinates is not None:
        for (x, y, r) in coordinates:
            # Tính toán tọa độ góc trên bên trái và góc dưới bên phải của hình chữ nhật
            x1 = x - r
            y1 = y - r
            
            x2 = x + r
            y2 = y + r
            cropped_image = image[y1:y2, x1:x2]
            label = meaning_of_signs(cropped_image)
            #  Vẽ hình chữ nhật lên hình ảnh gốc
            cv2.rectangle(image, (x1, y1), (x2, y2), color_rectangle[i_color], 5)
            i_color += 1
        
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_thickness = 1

            text_size, _ = cv2.getTextSize(label, font, font_scale, font_thickness)
            text_width, text_height = text_size

            text_height = text_height + 3

            # Tạo một mảng background_text màu trắng
            background_text = np.ones((text_height, text_width, 3), dtype="uint8") * 255

            # Viết nhãn trong hình chữ nhật
            cv2.putText(background_text, label, (0, text_height-2), font, font_scale, (0, 0, 255), font_thickness)

            height_image, width_image = image.shape[:2]

            if x1 + text_width > width_image:
                x1 = x1 - abs( x1 + text_width - width_image)
            if y2 + text_height > height_image:
                y2 = y2 - abs( y2 + text_height - height_image)

            # Chèn overlay vào góc trên bên trái của hình chữ nhật
            image[y2: y2+text_height, x1: x1 + text_width] = background_text

        # Hiển thị hình ảnh gốc và hình ảnh sau khi nhận diện hình tròn
        cv2.imwrite(f"Output/image_{index}.jpg", image)

In [70]:
i = 1
image = cv2.imread(f"Input/image_{i}.jpg")
while image is not None:
    draw_rectangle(image, i)
    i += 1
    image = cv2.imread(f"Input/image_{i}.jpg")

# Automatic training

In [71]:
def Automatic_training():
    i = 1
    image = cv2.imread(f"DataMining/image({i}).png")
    dataset = []
    while image is not None:
        coordinates = coordinates_detect(image)
        if coordinates is not None:
            for (x, y, r) in coordinates:
                x1 = x - r
                y1 = y - r
                x2 = x + r
                y2 = y + r

                cropped_image = image[y1:y2, x1:x2]
                label = meaning_of_signs(cropped_image)

                img_gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
                dataset.append({"image" : img_gray,"label" : label})
        i += 1
        image = cv2.imread(f"DataMining/image({i}).png")
    return dataset

In [72]:
# # Ghi dữ liệu vào file
# with open('data.pickle', 'ab') as file:
#     pickle.dump(Automatic_training(), file)